# GROUP BY Statements

### In this notebook we study some use case examples of the GROUP BY statments. We use Python as our main programming language and make use of libraries such as Pandas ans Psycopg2 to present the results of the queries.

# Aggregate Functions

Before we study the GROUP BY statements we must learn about aggregate functions. SQL provides us with a large number of aggregate function but we'll just cover a few of them here, the ones that I belive are the most common. 

The main idea behind the aggregate functions is to take several inputs and return a single input. In our particular case, we'll call an aggregate function on a given column (or a subset of the column) and it will return a single value made out of the values of the column.

In fact we´ve already seen an example of aggregate function. The COUNT() function takes a set of rows and returns the number of those rows.

In order to start coding, first, we have call the pandas and psycopg2 libraries and create a connection to communicate with the database.



In [1]:
import pandas as pd
import psycopg2 as pg2

connection = pg2.connect(database = 'dvdrental', user = 'postgres', password = 'password')

Just like in the prevous notebook, we´ll define the get_data function in order to prevent memory problems and to make our code more clean

In [2]:
def get_data(query, rows = 10):

    with connection.cursor() as cursor:
        cursor. execute(query)

        if rows == 'all':
            raw_data = cursor.fetchall()
        else:
            raw_data = cursor.fetchmany(rows) 

        col_names = [col_desc[0] for col_desc in cursor.description]
        data = pd.DataFrame(raw_data, columns = col_names)

    return data

## Ex. 1 (AVG)

We can use the AVG (average function) to calculate the average replacement cost of the movies in the database ## Ex. 1 (AVG)

We can use the AVG (average function) to calculate the average replacement cost of the movies in the database 

In [3]:
query_1 = 'SELECT AVG(replacement_cost) FROM film'
av_rc = get_data(query_1)

av_rc

,avg
0,19.9840000000000000


## Ex. 2 (MAX)

We can ask for the maximum replacement cost. 

In [4]:
query_2 = 'SELECT MAX(replacement_cost) FROM film'
max_rc = get_data(query_2)

max_rc 

,max
0,29.99


The case in which we ask for the minimum value is totally analogous, we just have to replace the MAX() function for the MIN() function. 

## Ex. 3 (SUM)

We use the SUM() function to obtain the sum of all the vlaues in a column. In the following, we calculate the sum of all the replacement costs

In [5]:
query_3 = 'SELECT SUM(replacement_cost) FROM film'
sum_rc = get_data(query_3)

sum_rc

,sum
0,19984.00


## Ex. 4

To conclude our brief discussion on aggregate functions, I would just like to add the fact that we can call several aggregate functions at the same time on different columns, for example## Ex. 4

To conclude our brief discussion on aggregate functions, I would just like to add the fact that we can call several aggregate functions at the same time on different columns, for example

In [6]:
query_4 = '''
          SELECT SUM(replacement_cost), AVG(length) 
          FROM film
          '''
custom_query = get_data(query_4)

custom_query

,sum,avg
0,19984.00,115.2720000000000000


# GROUP BY Statement

### The GROUP BY statement is used to group subsets of rows based on their values in a given field. The function performs this grouping by making use of the aggregate functions. The GROUP BY statement splits the rows into categories based on their value in a certain field and then it applies the aggregate function on each of those categories separately.

# Ex. 5

Suppose we want to know the total amount of money spent per customer. To achieve this we would have sum the amount of all the payments made by each of the customer separately. In other words we want to know the total amount per customer. The following lines of code carries on this task


In [7]:
query_5 = '''
          SELECT customer_id, SUM(amount) 
          FROM payment
          GROUP BY customer_id
          '''
amount_per_cust = get_data(query_5)

amount_per_cust

,customer_id,sum
0,184,80.80
1,87,137.72
2,477,106.79
3,273,130.72
4,550,151.69
5,51,123.70
6,394,77.80
7,272,65.87
8,70,75.83
9,190,102.75


# Ex. 6

If we wanted to know who are the five customers that have spent the most amount of money, we can make the following query

In [8]:
query_6 = '''
          SELECT customer_id, SUM(amount) 
          FROM payment
          GROUP BY customer_id 
          ORDER BY SUM(amount) DESC
          LIMIT 5
          '''
top_customers = get_data(query_6)

top_customers

,customer_id,sum
0,148,211.55
1,526,208.58
2,178,194.61
3,137,191.62
4,144,189.60


## Ex. 7

We can also group by multiple columns, for example, we could ask for the maximum amount spent per customer_id per staff_id and then order by customer_id and then we perform a suborder by staff_id

In [9]:
query_7 = '''
          SELECT customer_id, staff_id, SUM(amount) 
          FROM payment
          GROUP BY customer_id, staff_id
          ORDER BY customer_id, staff_id
          '''
max_amount = get_data(query_7)

max_amount

,customer_id,staff_id,sum
0,1,1,60.85
1,1,2,53.85
2,2,1,55.86
3,2,2,67.88
4,3,1,59.88
5,3,2,70.88
6,4,1,49.88
7,4,2,31.90
8,5,1,63.86
9,5,2,70.79


## To conclude this notebook we must close the connection with the database

In [10]:
connection.close()